In [1]:
!pip install hyperopt

In [17]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [7]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [8]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [9]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  model.fit(X_train, y_train)

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs',5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data', (X_train,y_train)),
      callbacks=[tensorboard_callback]
  )

  return model

def predict(model_trained,X_test,y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [39]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'), 
    Dropout(0.3),

    Dense(1024, activation='relu'), 
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
  ])


In [40]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

1088/1088 [==============================] - 6s 5ms/step - loss: 1.6765 - accuracy: 0.5080
Epoch 1/5
272/272 [==============================] - 8s 29ms/step - loss: 0.4155 - accuracy: 0.8667 - val_loss: 0.1225 - val_accuracy: 0.9632
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2624 - accuracy: 0.9193 - val_loss: 0.0667 - val_accuracy: 0.9803
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.1957 - accuracy: 0.9407 - val_loss: 0.0478 - val_accuracy: 0.9861
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.1646 - accuracy: 0.9522 - val_loss: 0.0314 - val_accuracy: 0.9911
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.1285 - accuracy: 0.9601 - val_loss: 0.0228 - val_accuracy: 0.9932


0.9648526077097506

In [36]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'), 
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'), 
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
  ])


In [37]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
 
  model.fit(
      X_train,
      y_train,
      batch_size=128,#int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )

  score  = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss':-accuracy, 'status': STATUS_OK,'model':model}

In [38]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.35283760370004524, 'dropout_cnn_block_three': 0.3721978220214431, 'dropout_cnn_block_two': 0.40667249547857587, 'dropout_dense_block_one': 0.6791164025359526, 'dropout_dense_block_two': 0.5028959279759957}
accuracy=0.9417233467102051
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.30421685715177205, 'dropout_cnn_block_three': 0.4531643973410042, 'dropout_cnn_block_two': 0.37592276979459266, 'dropout_dense_block_one': 0.6151914131318961, 'dropout_dense_block_two': 0.6972628026802767}
accuracy=0.9088435173034668
{'batch_size': 80.0, 'dropout_cnn_block_one': 0.3835407829608049, 'dropout_cnn_block_three': 0.3941370871347163, 'dropout_cnn_block_two': 0.3525529468486534, 'dropout_dense_block_one': 0.34328243788749635, 'dropout_dense_block_two': 0.4760495996204144}
accuracy=0.9306122660636902
{'batch_size': 80.0, 'dropout_cnn_block_one': 0.3010959530724433, 'dropout_cnn_block_three': 0.3124040057265824, 'dropout_cnn_block_two': 0.4509690847943